In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Introduction:

Cardiovascular diseases (CVDs) are thenumber 1 cause of death globally, taking an estimated 17.9 million lives each year, which accounts for 31% of all deaths worlwide.
Heart failure is a common event caused by CVDs and this dataset contains 12 features that can be used to predict mortality by heart failure.

In [ ]:
#1
df_hearth_failure = pd.read_csv('/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')
df_hearth_failure.head()

2) We are only going to use the feature columns 
“DEATH_EVENT, time, age, high_blood_pressure, ejection_fraction, serum_creatinine, serum_sodium”.


In [ ]:
df_hf = df_hearth_failure[['DEATH_EVENT', 'time', 'age', 'high_blood_pressure', 'ejection_fraction', 'serum_creatinine', 'serum_sodium']].copy()
df_hf.head()

3) Creating a new column called “age_text”, where the value is “low” for age<=56, “high” for age>= 73 and “medium” for ages between.

In [ ]:
age_text = lambda x: 'low' if(x<=56) else ('high' if(x>=73) else 'medium')
df_hf['age_text'] = df_hf['age'].apply(age_text)
df_hf.head()

3) remove the age column

In [ ]:
df_hf = df_hf.drop(['age'], axis=1)
df_hf.head()

4) Using the “from sklearn.preprocessing import OneHotEncoder” and doing a one hotencoding of “age_text” 


In [ ]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

encoder = OneHotEncoder(handle_unknown='ignore')

encoder.fit(np.c_[df_hf['age_text']])
encoder.categories

In [ ]:
transformed = encoder.transform(np.c_[df_hf['age_text']])
df_oh = pd.DataFrame(transformed.toarray())
df_oh.columns = ['high', 'low', 'medium']
df_oh.head()


4-b) Adding the three columns to the dataframe with the names “[age_text0, age_text1 and age_text2]

In [ ]:
df_hf[['age_text0', 'age_text1', 'age_text2']] = df_oh[['high', 'low', 'medium']]
df_hf = df_hf.drop(['age_text'], axis=1)
df_hf.head()

5) Using “from sklearn.preprocessing import StandardScaler” and scaling the columns “ejection_fraction, serum_creatinine, serum_sodium”.


Adding the columns back to the dataframe with the column names “ejection_fraction_sc, serum_creatinine_sc, serum_sodium_sc”, and removing the columns “ejection_fraction, serum_creatinine, serum_sodium”.

In [ ]:
df_hf[['ejection_fraction', 'serum_creatinine', 'serum_sodium']].describe()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df_hf[['ejection_fraction', 'serum_creatinine', 'serum_sodium']])
print(scaler.mean_)

In [ ]:
df_hf[['ejection_fraction_sc', 'serum_creatinine_sc', 'serum_sodium_sc']] = scaler.transform(df_hf[['ejection_fraction', 'serum_creatinine', 'serum_sodium']])
df_hf.head()

In [ ]:
askbjd = df_hf[['ejection_fraction', 'serum_creatinine', 'serum_sodium']]#saving just in case
df_hf = df_hf.drop(['ejection_fraction', 'serum_creatinine', 'serum_sodium'], axis=1)

6) Log scaling for any column where that seams desirable. 

In [ ]:
df_hf['serum_creatinine_sc_log'] = askbjd['serum_creatinine'].apply(np.log)
df_hf['serum_creatinine_sc_log'].hist()

In [ ]:
df_hf['serum_creatinine_sc'].hist()
askbjd['serum_creatinine_sc'] = df_hf['serum_creatinine_sc']
df_hf = df_hf.drop(['serum_creatinine_sc'], axis=1)

In [ ]:
df_hf.head()

<h1> Classification

From the prepped dataset above, extracting the column “DEATH_EVENT” as y, and “age_text0, age_text1, age_text2, high_blood_pressure, ejection_fraction_sc, serum_creatinine_sc, serum_sodium_sc” as X.

In [ ]:
df_hf.head()
y = df_hf['DEATH_EVENT']
X = np.c_[df_hf[['age_text0', 'age_text1', 'age_text2', 'high_blood_pressure', 'ejection_fraction_sc', 'serum_creatinine_sc_log', 'serum_sodium_sc']]]
print('success')

Using the “from sklearn.model_selection import train_test_split” and split X and y into X_train, X_test, y_train and y_test. Using a 20% test data size.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('success')


Train a logistic regression algorithm on the training data.

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(random_state=0)
log_reg.fit(X_train, y_train)

Make prediction using the training and test data, name the prediction variables y_train_pred and y_test_pred. 

Create a y_test_pred_naive which has the same shape as y_test, but only predicts the most common label.

In [ ]:
y_train_pred = log_reg.predict(X_train)
y_test_pred = log_reg.predict(X_test)
y_test_pred_naive = pd.Series([int(round(y.sum()/len(y))) for _ in range(len(y_test))])
print('predicted')

In [ ]:
len(y_test_pred_naive) == len(y_test)

<h1> 5
    


<p>Using “from sklearn.metricsimport accuracy_score” and calculate the train, test, and test_naive accuracy.****

In [ ]:
from sklearn.metrics import accuracy_score
print('TRAIN SCORE')
print(accuracy_score(y_train ,y_train_pred))

print('TEST SCORE')
print(accuracy_score(y_test ,y_test_pred))

print('NAIVE SCORE')
print(accuracy_score(y_test ,y_test_pred_naive))


Using “fromsklearn.metricsimportconfustion_matrix” and print the train, test, and test_naive confusion matrix.

In [ ]:
from sklearn.metrics import confusion_matrix
print('Format')
print('TN, FP')
print('FN, TP')
print('TRAIN CM SCORE')
print(confusion_matrix(y_train ,y_train_pred))
print('TEST CM SCORE')
print(confusion_matrix(y_test ,y_test_pred))

print('NAIVE CM SCORE')
print(confusion_matrix(y_test ,y_test_pred_naive))

The naive model got 35 correct predictions while our model got 41.
So our model is better.


Testing XGBoost

In [ ]:
import xgboost as xgb

X_trainxg, X_valid, y_trainxg, y_valid = train_test_split(X_train, y_train, test_size=0.05, random_state=42)


model=xgb.XGBClassifier(learning_rate=0.05, n_estimators=1000)
model.fit(X_trainxg, y_trainxg, eval_set=[(X_valid, y_valid)], early_stopping_rounds=2500, verbose=False)

In [ ]:
xg_pred = model.predict(X_test)
print(accuracy_score(xg_pred ,y_test))

Little bit better :)

<h1> Regression
   

From the prepped dataset above extract all columns, but only the rows where “DEATH_EVENT” is True (or 1).

In [ ]:
i_see_dead_people= df_hf[df_hf['DEATH_EVENT'] == 1].copy()
i_see_dead_people.head()

From the data with only death instances, extract the column “time” as y, 
and “age_text0, age_text1, age_text2, high_blood_pressure, ejection_fraction_sc, serum_creatinine_sc, serum_sodium_sc” as X.

In [ ]:
y = i_see_dead_people['time']
X = i_see_dead_people[['age_text0', 'age_text1', 'age_text2', 'high_blood_pressure', 'ejection_fraction_sc', 'serum_creatinine_sc_log', 'serum_sodium_sc']]
X.head()

Use the “np.c_[ ]” to get X and y in numpy format ready for training.

In [ ]:
X = np.c_[X]

Use the “from sklearn.model_selection import train_test_split” and split X and y into X_train_r, X_test_r, y_train_r and y_test_r. 

Using a 20% test data size.

In [ ]:
from sklearn.model_selection import train_test_split
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X, y, test_size=0.2, random_state=42)
print('success')

Train a linearregression algorithm on the training data

In [ ]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()

lr_model.fit(X_train_r, y_train_r)

Make prediction using the training and test data, name the prediction variables y_train_pred_r and y_test_pred_r. 


Create a y_test_pred_naive_r which has the same shape as y_test_r, but only predicts the average time for the patients who died.

In [ ]:
y_train_pred_r = lr_model.predict(X_train_r)
y_test_pred_r = lr_model.predict(X_test_r)
y_test_pred_naive_r = pd.Series([y_train_r.sum()/len(y_train_r) for _ in range(len(y_test_pred_r))])
print('Done')

Use “from sklearn.metrics import mean_square_error” and “from sklearn.metrics import mean_absolute_error” and calculate the train, test, and test_naive scores.



In [ ]:
from sklearn.metrics import mean_squared_error
print('TRAIN MSE')
print(mean_squared_error(y_train_r, y_train_pred_r))
print('TEST MSE')
print(mean_squared_error(y_test_r, y_test_pred_r))
print('NAIVE MSE')
print(mean_squared_error(y_test_r, y_test_pred_naive_r))


In [ ]:
from sklearn.metrics import mean_absolute_error
print('TRAIN MAE')
print(mean_absolute_error(y_train_r, y_train_pred_r))
print('TEST MAE')
print(mean_absolute_error(y_test_r, y_test_pred_r))
print('NAIVE MAE')
print(mean_absolute_error(y_test_r, y_test_pred_naive_r))


Conclusion: Our model is crap.


Try other models example k-nearest-neighbor, SVM. DecisionTrees, Xgboost, try hyperparameter tuning them. 

How good an MSE and MAE are you able to get on the test data?

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

NN = KNeighborsClassifier(n_neighbors=2, algorithm='ball_tree')
NN.fit(X_train_r, y_train_r)


In [ ]:
nny_train_pred_r = NN.predict(X_train_r)
nny_test_pred_r = NN.predict(X_test_r)
print('Done')

In [ ]:
from sklearn.metrics import mean_absolute_error
print('TRAIN MAE')
print(mean_absolute_error(y_train_r, nny_train_pred_r))
print('TEST MAE')
print(mean_absolute_error(y_test_r, nny_test_pred_r))
print('NAIVE MAE')
print(mean_absolute_error(y_test_r, y_test_pred_naive_r))

A little bit better.